<a href="https://colab.research.google.com/github/abulhasanat/NLP-Experiments/blob/master/text_generation_lstm(tf).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from string import punctuation
import keras

Using TensorFlow backend.


In [3]:
import requests

if not os.path.isdir("data"):
    os.mkdir("data")

content = requests.get("http://www.gutenberg.org/cache/epub/11/pg11.txt").text
open("data/wonderland.txt", "w", encoding="utf-8").write(content)

167516

In [4]:
sequence_length = 100
BATCH_SIZE = 128
EPOCHS = 30
# dataset file path
FILE_PATH = "data/wonderland.txt"
BASENAME = os.path.basename(FILE_PATH)
# read the data
text = open(FILE_PATH, encoding="utf-8").read()
# remove caps, comment this code if you want uppercase characters as well
text = text.lower()
# remove punctuation
text = text.translate(str.maketrans("", "", punctuation))

In [5]:
# print some stats
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 
 0123456789abcdefghijklmnopqrstuvwxyz﻿
Number of characters: 154861
Number of unique characters: 39


In [6]:
# dictionary that converts characters to integers
unique_chars=vocab
char2int = {c: i for i, c in enumerate(unique_chars)}
# dictionary that converts integers to characters
int2char = {i: c for i, c in enumerate(unique_chars)}

In [7]:
# save these dictionaries for later generation
pickle.dump(char2int, open(f"{BASENAME}-char2int.pickle", "wb"))
pickle.dump(int2char, open(f"{BASENAME}-int2char.pickle", "wb"))

In [8]:
# convert all text into integers
encoded_text = np.array([char2int[c] for c in text])

In [9]:
# construct tf.data.Dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [10]:
# print first 5 characters
for char in char_dataset.take(8):
    print(char.numpy(), int2char[char.numpy()])

38 ﻿
27 p
29 r
26 o
21 j
16 e
14 c
31 t


In [11]:
# build sequences by batching
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)

# print sequences
for sequence in sequences.take(2):
    print(''.join([int2char[i] for i in sequence.numpy()]))

﻿project gutenbergs alices adventures in wonderland by lewis carroll

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever  you may copy it give it away or

reuse it under the terms of the project gutenberg license included
with this ebook or online at wwwgutenbergorg


title alices adventures in wonderland

author lewis carroll

posting date june 25 2008


In [12]:
def split_sample(sample):
    ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
    for i in range(1, (len(sample)-1) // 2):
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
    return ds

# prepare inputs and targets
dataset = sequences.flat_map(split_sample)

In [13]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)


dataset = dataset.map(one_hot_samples)

In [14]:
# print first 2 samples
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: ﻿project gutenbergs alices adventures in wonderland by lewis carroll

this ebook is for the use of a
Target: n
Input shape: (100, 39)
Target shape: (39,)

Input: project gutenbergs alices adventures in wonderland by lewis carroll

this ebook is for the use of an
Target: y
Input shape: (100, 39)
Target shape: (39,)



In [15]:
# repeat, shuffle and batch the dataset
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [16]:
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),
])

In [17]:
optimizer = tf.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [18]:
# make results folder if does not exist yet
if not os.path.isdir("results"):
    os.mkdir("results")
# train the model
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=EPOCHS)
# save the model
model.save(f"results/{BASENAME}-{sequence_length}.h5")

Epoch 1/30
1209/1209 [==============================] - 45s 37ms/step - loss: 2.0633
Epoch 2/30
1209/1209 [==============================] - 44s 36ms/step - loss: 1.4949
Epoch 3/30
1209/1209 [==============================] - 44s 36ms/step - loss: 1.3776
Epoch 4/30
1209/1209 [==============================] - 44s 36ms/step - loss: 1.3174
Epoch 5/30
1209/1209 [==============================] - 44s 36ms/step - loss: 1.2779
Epoch 6/30
1209/1209 [==============================] - 44s 36ms/step - loss: 1.2506
Epoch 7/30
1209/1209 [==============================] - 44s 36ms/step - loss: 1.2269
Epoch 8/30
1209/1209 [==============================] - 43s 36ms/step - loss: 1.2118
Epoch 9/30
1209/1209 [==============================] - 43s 36ms/step - loss: 1.1912
Epoch 10/30
1209/1209 [==============================] - 44s 36ms/step - loss: 1.1772
Epoch 11/30
1209/1209 [==============================] - 45s 37ms/step - loss: 1.1664
Epoch 12/30
1209/1209 [==============================] - 45s 37

In [19]:
model.save(f'/content/drive/My Drive/{BASENAME}-{sequence_length}.h5')

# **Generating New Text**

In [20]:
import numpy as np
import pickle
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation
import os

sequence_length = 100
# dataset file path
FILE_PATH = "data/wonderland.txt"
# FILE_PATH = "data/python_code.py"
BASENAME = os.path.basename(FILE_PATH)

In [21]:
seed = "chapter xiii"
# load vocab dictionaries
char2int = pickle.load(open(f"{BASENAME}-char2int.pickle", "rb"))
int2char = pickle.load(open(f"{BASENAME}-int2char.pickle", "rb"))
vocab_size = len(char2int)

In [22]:
# building the model
model = Sequential([
    LSTM(256, input_shape=(sequence_length, vocab_size), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(vocab_size, activation="softmax"),
])

In [23]:
# load the optimal weights
model.load_weights(f"results/{BASENAME}-{sequence_length}.h5")

In [24]:
s = seed
n_chars = 400
# generate 400 characters
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
        X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|██████████| 400/400 [00:09<00:00, 41.02it/s]

Seed: chapter xiii
Generated text:
bbing and the mouse time and alice as she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and the stock and she spoket and
